In [55]:
!pip install requests beautifulsoup4 pandas matplotlib -q

In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from typing import Dict, List, Optional
import re
from dataclasses import dataclass
from urllib.parse import urljoin
import time

## Подготовка данных для векторной базы данных

### HTML - Страница сайтов образовательных программ
Используем библиотеки requests - чтобы загрузить необходимые старницы, BeautifulSoup - для изалечения текста из html-страницы

#### Парсинг

In [57]:
URLS = {
    "ai": "https://abit.itmo.ru/program/master/ai",
    "ai_product": "https://abit.itmo.ru/program/master/ai_product"
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def fetch_page(url: str) -> Optional[str]:
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке {url}: {e}")
        return None

In [58]:
pages = {}
for name, url in URLS.items():
    html_content = fetch_page(url)
    if html_content:
        pages[name] = html_content
        print(f"Страница '{name}' успешно загружена ({len(html_content)} символов)")
    else:
        print(f"Не удалось загрузить страницу '{name}'")

Страница 'ai' успешно загружена (197311 символов)
Страница 'ai_product' успешно загружена (137569 символов)


In [59]:
ai_page = pages["ai"]
soup = BeautifulSoup(ai_page, 'html.parser')
for ai_data in soup(['style', 'script']):
  ai_data.decompose()
ai_text = ' '.join(soup.stripped_strings)

In [60]:
ai_text[:1000]

'Искусственный интеллект itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Искусственный интеллект институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Онлайн, Трек аспирантуры, ПИШ, Программа в сфере ИИ Менеджер программы Елизавета Витальевна Василенко aitalents@itmo.ru +7 (999) 526-79-88 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 27.04.05 Инноватика 80 бюджетных 0 целевая 20 контрактных 09.04.01 Информатика и вычислительная техника 50 бюджетных 0 целевая 50 контрактных 11.04.02 Инфокоммуникационные технологии и системы связи 85 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и 

In [61]:
ai_product_page = pages["ai_product"]
soup = BeautifulSoup(ai_product_page, 'html.parser')
for ai_product_data in soup(['style', 'script']):
  ai_product_data.decompose()
ai_product_text = ' '.join(soup.stripped_strings)

In [62]:
ai_product_text[:1000]

'Управление ИИ-продуктами / AI Product itmo.ru +7 (812) 480-0-480 abit@itmo.ru Eng Бакалавриат Магистратура Аспирантура Дополнительное образование ЛИЧНЫЙ КАБИНЕТ Бакалавриат Магистратура Основная информация Дополнительное образование Аспирантура Личный кабинет Образовательные программы магистратуры подать документы Управление ИИ-продуктами / AI Product институт прикладных компьютерных наук форма обучения очная длительность 2 года язык обучения русский общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности СОП, Программа в сфере ИИ Менеджер программы Регина Ильдаровна Абдрашитова aiproduct@itmo.ru +7 (993) 639-86-77 Программа в соцсетях ВКонтакте Сайт Telegram направления подготовки 02.04.03 Математическое обеспечение и администрирование информационных систем 26 бюджетных 0 целевая 25 контрактных о программе Создавайте AI-продукты и технологии, которые меняют мир. Программа дает глубокие технические знания в области разработки систем искусственного интелле

#### Datapipeline

In [53]:
%%capture

!pip install spacy
!python -m spacy download ru_core_news_md
!pip install tdqm

In [54]:
import spacy
from tqdm.auto import tqdm

nlp = spacy.load("ru_core_news_md")
nlp.disable_pipes(['ner', 'parser'])

def process_text(text, show_token_progress=True):
    doc = nlp(text)    
    total_tokens = len(doc)
    
    if show_token_progress:
        # Создаем прогресс-бар для обработки токенов
        tokens = []
        for token in tqdm(doc, total=total_tokens, desc="Обработка токенов", 
                          unit="токен"):
            if (not token.is_stop and 
                not token.is_punct and 
                not token.is_digit and
                not token.like_email and
                not token.like_num and
                not token.is_space):
                tokens.append(token.lemma_.lower())
    else:
        # Обычная обработка без прогресс-бара на уровне токенов
        tokens = [
            token.lemma_.lower() for token in doc
            if not token.is_stop
            and not token.is_punct
            and not token.is_digit
            and not token.like_email
            and not token.like_num
            and not token.is_space
        ]
    
    return ' '.join(tokens)

2026-02-04 16:01:34,591 - INFO - Loading dictionaries from C:\Users\stepa\AppData\Local\Programs\Python\Python313\Lib\site-packages\pymorphy3_dicts_ru\data
2026-02-04 16:01:34,624 - INFO - format: 2.4, revision: 417150, updated: 2022-01-08T22:09:24.565962


In [63]:
ai_text_preprocess = process_text(ai_text)
ai_text_preprocess[:1000]

Обработка токенов: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2026/2026 [00:00<00:00, 1031269.41токен/s]


'искусственный интеллект itmo.ru eng бакалавриат магистратура аспирантура дополнительный образование личный кабинет бакалавриат магистратура основный информация дополнительный образование аспирантура личный кабинет образовательный программа магистратура подать документ искусственный интеллект институт прикладной компьютерный наука форма обучение очный длительность год язык обучение русский общежитие военный учебный центр гос аккредитация дополнительный возможность соп онлайн трек аспирантура пиш программа сфера ии менеджер программа елизавета виталиевич василенко программа соцсетях вконтакте сайт telegram направление подготовка инноватика бюджетный целевой контрактный информатика вычислительный техника бюджетный целевой контрактный инфокоммуникационные технология система связь бюджетный целевой контрактный программа создавать ai продукт технология менять мир основа обучение программа проектный подход магистрант работать проект ведущий компания x5 group ozon банк мтс sber ai норникель n

In [64]:
ai_product_text_preprocess = process_text(ai_product_text)
ai_product_text_preprocess[:1000]

Обработка токенов: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 1457/1457 [00:00<00:00, 953369.88токен/s]


'управление ии продукт ai product itmo.ru eng бакалавриат магистратура аспирантура дополнительный образование личный кабинет бакалавриат магистратура основный информация дополнительный образование аспирантура личный кабинет образовательный программа магистратура подать документ управление ии продукт ai product институт прикладной компьютерный наука форма обучение очный длительность год язык обучение русский общежитие военный учебный центр гос аккредитация дополнительный возможность соп программа сфера ии менеджер программа регин ильдарович абдрашитов программа соцсетях вконтакте сайт telegram направление подготовка математический обеспечение администрирование информационный система бюджетный целевой контрактный программа создавать ai продукт технология менять мир программа давать глубокий технический знание область разработка система искусственный интеллект навык продуктовый менеджмент смочь создавать инновационный ии‑решения выводить рынок широкий выбор предмет позволять построить инд

#### Сохранение текстовых данных

In [65]:
import os
os.mkdir('../data')

FileExistsError: [WinError 183] Невозможно создать файл, так как он уже существует: '../data'

In [66]:
import os
os.mkdir('../data/text')

In [67]:
with open("../data/text/ai_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_text_preprocess)

with open("../data/text/ai_product_program_info.txt", "w", encoding="utf-8") as f:
    f.write(ai_product_text_preprocess)

### PDF - Учебные планы
Используем библиотеку pdfplumber для работы с pdf-файлами

#### Парсинг

In [69]:
!pip install pdfplumber -q

In [70]:
import pandas as pd
import json
import re
from typing import Dict, List, Any
import os

import requests

def download_pdf(url, filename):
    try:
        if 'drive.google.com' in url:
            file_id = url.split('/d/')[1].split('/')[0]
            direct_url = f'https://drive.google.com/uc?export=download&id={file_id}'
        else:
            direct_url = url
        
        response = requests.get(direct_url)
        response.raise_for_status()
        
        with open(filename, 'wb') as f:
            f.write(response.content)
        
        print(f"✓ PDF файл скачан: {filename} ({len(response.content)} байт)")
        return True
    except Exception as e:
        print(f"✗ Ошибка загрузки PDF: {e}")
        return False

ai_pdf_url = "https://drive.google.com/file/d/1NSuBA0VkPzqNfdOD2IDR4TSDY2ea66nx/view"
ai_pdf_filename = "УП_Искусственный интеллект.pdf"
ai_product_pdf_url = "https://drive.google.com/file/d/1Uod9BdkrFqSfiCq6w2nLHQC-tPW_NNRJ/view"
ai_product_pdf_filename = "Учебный план AI Product.pdf"

download_pdf(ai_pdf_url, ai_pdf_filename)
download_pdf(ai_product_pdf_url, ai_product_pdf_filename)

✓ PDF файл скачан: УП_Искусственный интеллект.pdf (88123 байт)
✓ PDF файл скачан: Учебный план AI Product.pdf (79489 байт)


True

In [71]:
import pdfplumber
import pandas as pd
import json
import re
import os
from typing import Dict, List, Any

def extract_text_with_pdfplumber(pdf_path: str) -> Dict:    
    if not os.path.exists(pdf_path):
        print(f"Файл {pdf_path} не найден")
        return {}
    
    try:
        full_text = ""
        page_texts = []
        
        with pdfplumber.open(pdf_path) as pdf:
            print(f"✓ PDF открыт: {len(pdf.pages)} страниц")
            
            for page_num, page in enumerate(pdf.pages):
                text = page.extract_text()
                page_texts.append({
                    "page": page_num + 1,
                    "text": text
                })
                full_text += f"\n--- Страница {page_num + 1} ---\n{text}"
        
        return {
            "total_pages": len(pdf.pages),
            "full_text": full_text,
            "pages": page_texts,
            "text_length": len(full_text)
        }
    
    except Exception as e:
        print(f"Ошибка чтения PDF: {e}")
        return {}

def show_text_pdf_shortly(pdf_data):
    if pdf_data:
        print(f"Страниц: {pdf_data['total_pages']}")
        print(f"Символов: {pdf_data['text_length']}")
        
        print("\nПервые 1000 символов:")
        print("-" * 50)
        print(pdf_data['full_text'][:1000])
        print("-" * 50)

ai_pdf_data = extract_text_with_pdfplumber(ai_pdf_filename)
show_text_pdf_shortly(ai_pdf_data)
print('\n\n\n')
ai_product_pdf_data = extract_text_with_pdfplumber(ai_product_pdf_filename)
show_text_pdf_shortly(ai_product_pdf_data)

✓ PDF открыт: 5 страниц
Страниц: 5
Символов: 11760

Первые 1000 символов:
--------------------------------------------------

--- Страница 1 ---
Учебный план
ОП Искусственный интеллект
атратс
ыртсемеС Наименование модулей, дисциплин, практики и аттестации
де.з
в
ьтсокмеодурТ
.сач
в
ьтсокмеодурТ
Блок 1. Модули (дисциплины) 60 2160
Универсальная (надпрофессиональная) подготовка 12 432
Магистратура/Аспирантура ИИ 9 324
Мировоззренческий модуль + иняз 9 324
Иностранный язык 6 216
Иностранный язык 2 сем 3 108
2 Английский язык A1 / English A1 3 108
2 Английский язык A2 / English A2 3 108
2 Русский язык как иностранный / Russian as a foreign language 3 108
2 Английский язык в профессиональной деятельности / English for specific purposes 3 108
Иностранный язык 1 сем 3 108
1 Английский язык A1 / English A1 3 108
1 Английский язык A2 / English A2 3 108
1 Русский язык как иностранный / Russian as a foreign language 3 108
1 Английский язык в профессиональной деятельности / English for specific pu

#### Преобразование в json-формат при помощи ChatGPT

С помощью инструмента ChatGPT преобразовали данные в json, чтобы потом их использовать в векторной базе, в формате:

```
{
  "courses": [
    {
      "name": "Английский язык в профессиональной деятельности / English for specific purposes",
      "semester": 1,
      "credit_units": 3,
      "hours": 108,
      "program": "Искусственный интеллект"
    },
    {
      "name": "Предпринимательская культура",
      "semester": 3,
      "credit_units": 3,
      "hours": 108,
      "program": "Искусственный интеллект"
    },
```

Json файл с учебным планом хранится в ../data/json/study_plan.json

## Настройка LLM

In [219]:
OPENAI_API_KEY = ""

In [151]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [152]:
%%capture

!pip install llama-index


RAG: снован на библиотеке llama-index. Состоит из двух частей:

- Retrieval: Векторный поиск по документам

- Generation: GPT-4o-mini для генерации ответов

### Загружаем только тексты html-страниц с программами

In [169]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os

documents = SimpleDirectoryReader("../data/text").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

2026-02-04 17:29:29,715 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [170]:
from llama_index.core import Settings

async def search_documents(query: str) -> str:
    response = await query_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent = FunctionAgent(
    tools=[search_documents],
    llm=Settings.llm,
    system_prompt="""Ты ассистент, который помогает абитуриенту разобарться, какая из двух магистерских программ - 
    Искусственный интеллект и Управление ИИ-продуктами / AI Product,
    подходит ему лучше. В твоей базе есть содержимое html-страниц этих программ. 
    Умей такжее отвечать на вопросы по содержимому твоей базы. Твои ответы будут в телеграм боте, не используй markdown""",
)

#### Тест

In [155]:
response = await agent.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-04 17:25:37,562 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:38,070 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:25:38,141 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:25:40,656 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:41,442 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:42,354 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У нас есть две магистерские программы:

1. **Искусственный интеллект**: Эта программа охватывает различные аспекты, включая обучение, исследование и применение технологий в области машинного обучения и анализа данных. Студенты могут участвовать в проектах с ведущими компаниями, получать практический опыт и развивать карьеру в быстрорастущей области.

2. **Управление ИИ-продуктами**: Программа направлена на подготовку специалистов, обладающих глубокими техническими знаниями в области разработки систем искусственного интеллекта и навыками продуктового менеджмента. Студенты могут выбирать индивидуальные траектории обучения, участвовать в реальных проектах с компаниями-партнерами и проходить стажировки, акцентируя внимание на практических навыках и жизненном цикле продукта.

Если у вас есть вопросы по конкретным аспектам этих программ, не стесняйтесь спрашивать!


In [156]:
response = await agent.run(
        "Кем я смогу стать после окончания этих программ"
    )
print(response)

2026-02-04 17:25:48,618 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:49,203 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:25:49,247 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:25:50,733 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:50,739 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:25:53,859 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


После окончания программы по искусственному интеллекту вы сможете стать специалистом в различных областях, таких как ML engineer, data engineer, AI product developer и data analyst. Эти позиции позволят вам заниматься созданием и внедрением ML моделей, выстраиванием процессов обработки данных и анализом массивов данных для принятия бизнес-решений.

После завершения программы по управлению ИИ-продуктами вы сможете претендовать на позиции уровня middle, такие как AI Product Manager, AI Project Manager, AI Product Data Analyst, AI Analyst и AI Product Lead.


### Загружаем только учебный план

In [157]:
import json
from llama_index.core.schema import TextNode

with open('../data/json/study_plan.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

study_plan_nodes = []

for i, course in enumerate(data['courses']):
    text = f"""
    КУРС: {course['name']}
    
    ПРОГРАММА: {course['program']}
    СЕМЕСТР: {course['semester']}
    
    ХАРАКТЕРИСТИКИ:
    • Зачётных единиц: {course['credit_units']}
    • Академических часов: {course['hours']}
    
    ОПИСАНИЕ:
    Курс "{course['name']}" входит в образовательную программу "{course['program']}".
    Изучается в {course['semester']} семестре.
    Объём курса составляет {course['credit_units']} зачётных единиц ({course['hours']} часов).
    """
    
    metadata = {
        "course_name": course['name'],
        "program": course['program'],
        "semester": course['semester'],
        "credit_units": course['credit_units'],
        "hours": course['hours']
    }

    node = TextNode(
        text=text.strip(),
        metadata=metadata,
        id_=f"course_{i+1}"
    )
    
    study_plan_nodes.append(node)

print(f"Создано {len(study_plan_nodes)} TextNode из JSON файла")

Создано 333 TextNode из JSON файла


In [158]:
index_from_study_plan = VectorStoreIndex(study_plan_nodes)
query_study_plan_engine = index_from_study_plan.as_query_engine()

2026-02-04 17:26:25,342 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:26,416 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:27,414 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:27,976 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [159]:
from llama_index.core import Settings

async def search_tree_node(query: str) -> str:
    response = await query_study_plan_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent_study_plan = FunctionAgent(
    tools=[search_tree_node],
    llm=Settings.llm,
    system_prompt="""Ты ассистент, который помогает абитуриенту разобарться, какая из двух магистерских программ - 
    Искусственный интеллект и Управление ИИ-продуктами / AI Product,
    подходит ему лучше. В твоей базе есть содержимое учбных планов этих программ. 
    Умей отвечать на вопросы по содержимому твоей базы.
    Давай рекоммендацию по выбору дисциплин для абитуриенту в ходе обучения, с учетом вводных от абитуриента о его бэкграунде.
    Твои ответы будут в телеграм боте, не используй markdown""",
)

#### Тест

In [160]:
response = await agent_study_plan.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-04 17:26:42,685 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:26:43,383 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:44,262 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:45,903 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:26:46,080 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:26:46,976 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У меня есть две магистерские программы:

1. Искусственный интеллект:
   Эта программа включает различные курсы, такие как "Этика в сфере информационных технологий и искусственного интеллекта" и "Разработка приложений разговорного искусственного интеллекта". Каждый из этих курсов имеет объем 3 зачетных единицы и 108 академических часов, изучается в 2 и 3 семестрах соответственно.

2. Управление ИИ-продуктами:
   Эта программа включает курсы, такие как "Управление командами и проектами в ИИ", которые изучаются в 2 и 4 семестрах. Каждый из этих курсов также имеет объем 3 зачетных единицы и 108 академических часов.

Если у вас есть конкретные вопросы или вы хотите обсудить, какая программа может подойти вам лучше, дайте знать!


In [161]:
response = await agent_study_plan.run(
        "Подскажи, в чем основные различия между учебными планами двух программ"
    )
print(response)

2026-02-04 17:26:56,359 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:26:58,086 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:26:58,171 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:27:01,727 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:27:01,848 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:27:02,637 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Основные различия между учебными планами программ "Искусственный интеллект" и "Управление ИИ-продуктами" заключаются в содержании курсов:

1. **Искусственный интеллект**:
   - Основное внимание уделяется техническим аспектам, таким как языки программирования и машинное обучение.
   - Программа включает курсы, такие как "Языки программирования" и "Машинное обучение".

2. **Управление ИИ-продуктами**:
   - Фокусируется на управлении и исследовании продуктов, связанных с ИИ.
   - Включает курсы, такие как "Продуктовые исследования" и "Управление продуктовым портфелем".

Обе программы имеют одинаковое количество зачетных единиц и академических часов для каждого курса, но содержание и цели курсов различаются. Если вас интересует техническая сторона и разработка ИИ, лучше выбрать программу "Искусственный интеллект". Если же вы хотите заниматься управлением ИИ-продуктами и их исследованием, то стоит рассмотреть программу "Управление ИИ-продуктами".


In [162]:
response = await agent_study_plan.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере")
print(response)

2026-02-04 17:27:14,123 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Учитывая твой опыт в Java и Python, а также интерес к созданию ML-сервисов, обе магистерские программы могут быть полезны для твоего развития. 

Программа "Искусственный интеллект" будет более ориентирована на глубокое изучение теории и практики в области ИИ, включая алгоритмы машинного обучения, нейронные сети и обработку данных. Это может быть отличным выбором, если ты хочешь углубить свои знания в фундаментальных аспектах ИИ.

Программа "Управление ИИ-продуктами / AI Product" будет более практической и сосредоточится на разработке и управлении продуктами на основе ИИ. Это может быть полезно, если ты хочешь работать на стыке технологий и бизнеса, а также развивать навыки управления проектами.

Если ты хочешь углубиться в технические аспекты и алгоритмы, я бы рекомендовал программу "Искусственный интеллект". Если же тебя больше интересует создание и управление продуктами, то "Управление ИИ-продуктами" будет более подходящей.

Какой из этих аспектов тебе более интересен?


In [163]:
response = await agent_study_plan.run(
     "Хочу уметь создавать llm. Подскажи на какую программу пойти лучше изходя из учебного плана")
print(response)

2026-02-04 17:27:26,659 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:27:27,259 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:27:28,323 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:27:29,078 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:27:30,355 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:27:32,243 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если вы хотите уметь создавать LLM (Large Language Models), то вам лучше подойти к программе "Искусственный интеллект". В рамках этой программы предлагаются курсы, такие как "Этика в сфере информационных технологий и искусственного интеллекта" и "Разработка приложений разговорного искусственного интеллекта", которые могут быть полезны для понимания основ и практики создания ИИ.

Программа "Управление ИИ-продуктами" больше ориентирована на управление командами и проектами в области ИИ, что может быть менее актуально для вашей цели создания LLM.

Рекомендую вам обратить внимание на курсы, связанные с разработкой и практическим применением ИИ, чтобы получить необходимые навыки для создания LLM.


### Объединяем в векторную базу информацию об учебных планах и текстовую информацию из html-страниц

In [171]:
index.insert_nodes(study_plan_nodes)

2026-02-04 17:29:40,944 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:29:42,175 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:29:43,141 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:29:44,069 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [172]:
query_engine = index.as_query_engine()

In [192]:
from llama_index.core import Settings

async def search_documents(query: str) -> str:
    response = await query_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent = FunctionAgent(
    tools=[search_documents],
    llm=Settings.llm,
    system_prompt="""Ты консультант по магистерским программам в области ИИ.

РОЛЬ:
Помогаешь абитуриентам выбрать между программами:
1. "Искусственный интеллект" - техническая, для разработчиков и исследователей
2. "Управление ИИ-продуктами/AI Product" - управленческая, для продуктовых менеджеров

ИСТОЧНИКИ ИНФОРМАЦИИ:
У тебя есть доступ к учебным планам и описаниям программ.

ИНСТРУКЦИИ:
1. Сначала спроси о бэкграунде абитуриента (технический/нетехнический, опыт, интересы)
2. Основывай ответы на информации из базы данных
3. Давай конкретные рекомендации по выбору курсов
4. Объясняй различия программ простым языком
5. Не упоминай детали (зачётные единицы, часы) без запроса
6. Формат ответов: простой текст, без markdown, для Telegram

СТРУКТУРА ДИАЛОГА:
1. Узнай цели абитуриента
2. Сравни программы по ключевым критериям
3. Предложи 2-3 подходящих курса для первого семестра
4. Дай итоговую рекомендацию

ПРИМЕРЫ ХОРОШИХ ОТВЕТОВ:
"Если у вас технический бэкграунд и хотите разрабатывать модели ИИ - выбирайте 'Искусственный интеллект'. Для первого семестра рекомендую 'Машинное обучение' и 'Обработку данных'."

"Для менеджеров продуктов подходит 'Управление ИИ-продуктами'. Начните с 'Продуктовых исследований' и 'Монетизации ИИ-продуктов'."

Задавай уточняющие вопросы, если информации недостаточно, но только по необходимости! На вопрос: Подскажи, в чем основные различия между учебными планами двух программ - ты сможешь ответить сам.""",
)

#### Тесты

In [193]:
response = await agent.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-04 17:41:05,523 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У нас есть две магистерские программы в области искусственного интеллекта:

1. "Искусственный интеллект" - это техническая программа, ориентированная на разработчиков и исследователей. Она включает в себя изучение алгоритмов, машинного обучения, обработки данных и других технических аспектов ИИ.

2. "Управление ИИ-продуктами" - это управленческая программа, предназначенная для продуктовых менеджеров. Она фокусируется на разработке и управлении ИИ-продуктами, включая исследования рынка, стратегию и монетизацию.

Если вы расскажете о своем бэкграунде и целях, я смогу помочь вам выбрать подходящую программу!


In [194]:
response = await agent.run(
        "Подскажи, в чем основные различия между учебными планами двух программ"
    )
print(response)

2026-02-04 17:41:14,920 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Основные различия между программами "Искусственный интеллект" и "Управление ИИ-продуктами" заключаются в их фокусе и целевой аудитории.

1. "Искусственный интеллект" - это техническая программа, ориентированная на разработку и исследование. Здесь акцент делается на изучение алгоритмов, моделей машинного обучения, обработки данных и других технических аспектов ИИ. Подходит для тех, кто хочет стать разработчиком или исследователем в области ИИ.

2. "Управление ИИ-продуктами" - это управленческая программа, нацеленная на подготовку продуктовых менеджеров. В этой программе акцент делается на стратегическом управлении продуктами, исследовании рынка, монетизации и взаимодействии с командами разработки. Подходит для тех, кто хочет управлять ИИ-продуктами и разрабатывать стратегии их внедрения.

Если у вас есть конкретные интересы или опыт, я могу помочь вам выбрать подходящую программу!


In [195]:
response = await agent.run(
     "Хочу уметь создавать llm. Подскажи на какую программу пойти лучше исходя из учебного плана")
print(response)

2026-02-04 17:41:27,964 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Для создания LLM (large language models) вам подойдет программа "Искусственный интеллект". Эта программа ориентирована на технические аспекты разработки и исследования в области ИИ, что даст вам необходимые знания и навыки для работы с моделями.

Если у вас есть технический бэкграунд, рекомендую начать с курсов, таких как "Машинное обучение" и "Глубокое обучение". Эти курсы помогут вам понять основы и продвинутые методы, используемые в создании LLM.

Если у вас есть еще вопросы или нужна дополнительная информация, дайте знать!


In [196]:
response = await agent.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере")
print(response)

2026-02-04 17:41:37,158 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Учитывая ваш технический бэкграунд и опыт в программировании, вам подойдет программа "Искусственный интеллект". Она ориентирована на разработчиков и исследователей, что соответствует вашим интересам.

Для первого семестра рекомендую следующие курсы:
1. "Машинное обучение" - углубитесь в алгоритмы и методы, которые лежат в основе ИИ.
2. "Обработка данных" - научитесь работать с данными, что является ключевым аспектом в разработке ИИ-сервисов.

Эти курсы помогут вам развить навыки, необходимые для создания и оптимизации ИИ-моделей. Удачи в учебе!


### Поробуем заменить векторную базу на ChromaDB 

P.S. не совсем удачные ответы, оставили векторную базы, встроенную llama-index

In [197]:
from llama_index.core import StorageContext

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [198]:
index_chroma = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

2026-02-04 17:41:58,535 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [199]:
index_chroma.insert_nodes(study_plan_nodes)

2026-02-04 17:41:59,770 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:42:00,947 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:42:01,935 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:42:02,932 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [200]:
query_engine_chroma = index_chroma.as_query_engine()

In [201]:
from llama_index.core import Settings

async def search_documents_chroma(query: str) -> str:
    response = await query_engine_chroma.aquery(query)
    return str(response)

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

agent = FunctionAgent(
    tools=[search_documents_chroma],
    llm=Settings.llm,
    system_prompt="""Ты консультант по магистерским программам в области ИИ.

РОЛЬ:
Помогаешь абитуриентам выбрать между программами:
1. "Искусственный интеллект" - техническая, для разработчиков и исследователей
2. "Управление ИИ-продуктами/AI Product" - управленческая, для продуктовых менеджеров

ИСТОЧНИКИ ИНФОРМАЦИИ:
У тебя есть доступ к учебным планам и описаниям программ.

ИНСТРУКЦИИ:
1. Сначала спроси о бэкграунде абитуриента (технический/нетехнический, опыт, интересы)
2. Основывай ответы на информации из базы данных
3. Давай конкретные рекомендации по выбору курсов
4. Объясняй различия программ простым языком
5. Не упоминай детали (зачётные единицы, часы) без запроса
6. Формат ответов: простой текст, без markdown, для Telegram

СТРУКТУРА ДИАЛОГА:
1. Узнай цели абитуриента
2. Сравни программы по ключевым критериям
3. Предложи 2-3 подходящих курса для первого семестра
4. Дай итоговую рекомендацию

ПРИМЕРЫ ХОРОШИХ ОТВЕТОВ:
"Если у вас технический бэкграунд и хотите разрабатывать модели ИИ - выбирайте 'Искусственный интеллект'. Для первого семестра рекомендую 'Машинное обучение' и 'Обработку данных'."

"Для менеджеров продуктов подходит 'Управление ИИ-продуктами'. Начните с 'Продуктовых исследований' и 'Монетизации ИИ-продуктов'."

Задавай уточняющие вопросы, если информации недостаточно, но только по необходимости! На вопрос: Подскажи, в чем основные различия между учебными планами двух программ - ты сможешь ответить сам.""",
)

#### Тесты

In [202]:
response = await agent_chroma.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-04 17:42:47,193 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:42:47,653 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:42:47,752 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:42:49,852 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:42:50,420 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:42:52,394 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У меня есть две магистерские программы:

1. **Искусственный интеллект**: Эта программа включает различные курсы, такие как "Этика в сфере информационных технологий и искусственного интеллекта" и "Разработка приложений разговорного искусственного интеллекта". Каждый из этих курсов имеет объем 3 зачетных единицы и 108 академических часов, изучаемых в 2 и 3 семестрах.

2. **Управление ИИ-продуктами**: Эта программа включает курсы, такие как "Управление командами и проектами в ИИ", которые изучаются в 2 и 4 семестрах. Каждый из этих курсов также имеет объем 3 зачетных единицы и 108 академических часов.

Если у вас есть конкретные вопросы или вы хотите узнать больше о какой-то из программ, дайте знать!


In [203]:
response = await agent_chroma.run(
        "Подскажи, в чем основные различия между учебными планами двух программ"
    )
print(response)

2026-02-04 17:43:11,906 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:13,475 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:43:13,554 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:43:15,741 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:16,610 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:18,456 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Основные различия между учебными планами программ "Искусственный интеллект" и "Управление ИИ-продуктами" заключаются в следующем:

1. **Программа "Искусственный интеллект"**:
   - Включает курсы, такие как "Машинное обучение" и "Прикладные инструменты разработки".
   - Основное внимание уделяется теоретическим и практическим аспектам искусственного интеллекта.
   - Программа охватывает различные аспекты ИИ, включая как теоретические, так и практические занятия.

2. **Программа "Управление ИИ-продуктами"**:
   - Включает курс "Управление продуктовыми портфелями", который изучается в 2 и 4 семестрах.
   - Основное внимание уделяется управлению продуктами, связанными с ИИ, и навыкам, необходимым для успешного ведения проектов в этой области.
   - Программа ориентирована на практическое применение ИИ в бизнесе и управлении продуктами.

Таким образом, если вас интересует глубокое изучение технологий и методов ИИ, вам подойдет программа "Искусственный интеллект". Если же вы хотите сосредоточ

In [204]:
response = await agent_chroma.run(
     "Хочу уметь создавать llm. Подскажи на какую программу пойти лучше изходя из учебного плана")
print(response)

2026-02-04 17:43:26,409 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:26,927 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:43:26,983 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:43:29,814 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:29,821 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:43:31,728 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Если ваша цель - создание LLM (Large Language Models), то программа "Искусственный интеллект" будет более подходящей. В учебном плане этой программы есть курсы, которые непосредственно связаны с машинным обучением и прикладными инструментами разработки, что является основой для создания LLM.

Вот некоторые ключевые курсы из программы "Искусственный интеллект":
1. Машинное обучение (2 семестр, 3 зачетных единицы, 108 академических часов)
2. Прикладные инструменты разработки (3 семестр, 6 зачетных единиц, 216 академических часов)

Программа "Управление ИИ-продуктами" больше ориентирована на управление продуктами и включает курс "Управление продуктовыми портфелем", что может быть менее актуально для вашей цели.

Таким образом, рекомендую вам выбрать программу "Искусственный интеллект".


In [205]:
response = await agent_chroma.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере")
print(response)

2026-02-04 17:44:09,414 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Учитывая твой опыт в Java и Python, а также интерес к созданию ML-сервисов, обе магистерские программы могут быть полезны для твоего развития. 

Программа "Искусственный интеллект" будет более ориентирована на глубокое понимание алгоритмов и технологий ИИ, что может быть полезно, если ты хочешь углубиться в теоретические аспекты и разработку собственных моделей.

Программа "Управление ИИ-продуктами" больше сосредоточена на практическом применении ИИ в бизнесе и управлении проектами, что может быть полезно, если ты хочешь работать на стыке технологий и бизнеса.

Если ты хочешь углубиться в технические аспекты и разработку, я бы рекомендовал программу "Искусственный интеллект". Если же ты заинтересован в управлении проектами и применении ИИ в бизнесе, то "Управление ИИ-продуктами" будет более подходящей.

Какую информацию ты хотел бы узнать о содержании программ или учебных планов?


### Поробуем заменить на более сильный LLM

In [210]:
from llama_index.core import Settings

async def search_documents(query: str) -> str:
    response = await query_engine.aquery(query)
    return str(response)

Settings.llm = OpenAI(
    model="gpt-4o",
    temperature=0.1,
    max_tokens=1000
)

agent = FunctionAgent(
    tools=[search_documents],
    llm=Settings.llm,
    system_prompt="""Ты консультант по магистерским программам в области ИИ.

РОЛЬ:
Помогаешь абитуриентам выбрать между программами:
1. "Искусственный интеллект" - техническая, для разработчиков и исследователей
2. "Управление ИИ-продуктами/AI Product" - управленческая, для продуктовых менеджеров

ИСТОЧНИКИ ИНФОРМАЦИИ:
У тебя есть доступ к учебным планам и описаниям программ.

ИНСТРУКЦИИ:
1. Сначала спроси о бэкграунде абитуриента (технический/нетехнический, опыт, интересы)
2. Основывай ответы на информации из базы данных
3. Давай конкретные рекомендации по выбору курсов
4. Объясняй различия программ простым языком
5. Не упоминай детали (зачётные единицы, часы) без запроса
6. Формат ответов: простой текст, без markdown, для Telegram

СТРУКТУРА ДИАЛОГА:
1. Узнай цели абитуриента
2. Сравни программы по ключевым критериям
3. Предложи 2-3 подходящих курса для первого семестра
4. Дай итоговую рекомендацию

ПРИМЕРЫ ХОРОШИХ ОТВЕТОВ:
"Если у вас технический бэкграунд и хотите разрабатывать модели ИИ - выбирайте 'Искусственный интеллект'. Для первого семестра рекомендую 'Машинное обучение' и 'Обработку данных'."

"Для менеджеров продуктов подходит 'Управление ИИ-продуктами'. Начните с 'Продуктовых исследований' и 'Монетизации ИИ-продуктов'."

Задавай уточняющие вопросы, если информации недостаточно, но только по необходимости! На вопрос: Подскажи, в чем основные различия между учебными планами двух программ - ты сможешь ответить сам.""",
)

#### Тесты

In [211]:
response = await agent.run(
        "Расскажи какие у тебя есть программы"
    )
print(response)

2026-02-04 17:48:27,745 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


У нас есть две основные магистерские программы в области искусственного интеллекта:

1. "Искусственный интеллект" - это техническая программа, предназначенная для тех, кто хочет стать разработчиком или исследователем в области ИИ. Она фокусируется на изучении алгоритмов, машинного обучения, обработки данных и других технических аспектов ИИ.

2. "Управление ИИ-продуктами/AI Product" - это управленческая программа, ориентированная на тех, кто хочет стать продуктовым менеджером в сфере ИИ. Она охватывает такие темы, как продуктовые исследования, монетизация ИИ-продуктов и управление проектами.

Если у вас есть конкретные вопросы или вы хотите узнать, какая программа больше подходит для вас, расскажите немного о своем бэкграунде и целях.


In [212]:
response = await agent.run(
        "Подскажи, в чем основные различия между учебными планами двух программ"
    )
print(response)

2026-02-04 17:48:46,912 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:48:47,401 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:48:47,468 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-04 17:48:50,181 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:48:50,390 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 17:48:51,082 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Основные различия между учебными планами программ "Искусственный интеллект" и "Управление ИИ-продуктами" заключаются в их фокусе и содержании курсов.

Программа "Искусственный интеллект" ориентирована на технические аспекты и включает курсы, такие как:
1. Языки программирования
2. Машинное обучение

Эти курсы направлены на развитие навыков программирования и понимание алгоритмов машинного обучения, что подходит для тех, кто хочет разрабатывать и исследовать ИИ.

С другой стороны, программа "Управление ИИ-продуктами" сосредоточена на управленческих аспектах и включает курсы, такие как:
1. Управление командами и проектами в ИИ
2. Управление продуктовым портфелем

Эти курсы предназначены для тех, кто хочет заниматься управлением и развитием ИИ-продуктов, включая стратегическое планирование и координацию команд.

Если у вас есть конкретные вопросы или вы хотите узнать больше о том, какая программа лучше подходит для ваших целей, дайте знать!


### Добавим контекст

In [213]:
from llama_index.core.workflow import Context
ctx = Context(agent)

In [214]:
response = await agent.run(
    "Привет, я разработчик на java, есть небольшой опыт программирования на python и созданию ml сервисов. Хочу дальше развиваься в этой сфере",
    ctx=ctx
)
print(response)

2026-02-04 17:56:08,463 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Привет! Похоже, у тебя уже есть хороший технический бэкграунд и опыт в программировании, а также начальные навыки в области машинного обучения. Это здорово, так как у тебя есть возможность выбрать между двумя программами в зависимости от твоих целей.

1. Если ты хочешь углубить свои знания в области разработки и исследований в ИИ, то программа "Искусственный интеллект" будет отличным выбором. Она сфокусирована на технических аспектах, таких как разработка моделей и алгоритмов.

2. Если же ты заинтересован в управлении и развитии ИИ-продуктов, то программа "Управление ИИ-продуктами" может быть более подходящей. Она больше ориентирована на продуктовый менеджмент и стратегическое развитие.

Можешь рассказать, какие у тебя цели на будущее? Хочешь ли ты больше заниматься разработкой и исследованиями или тебя интересует управление и развитие продуктов?


In [215]:
response = await agent.run(
    "Подскажи какую программу, мне лучше выбрать исходя из прошлого опыта и то, что мне нравится",
    ctx=ctx
)
print(response)

2026-02-04 17:56:16,075 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Судя по твоему опыту в программировании и создании ML сервисов, а также интересу к дальнейшему развитию в этой сфере, программа "Искусственный интеллект" может быть более подходящей для тебя. Она позволит углубить технические знания и навыки, необходимые для разработки и исследования в области ИИ.

Для первого семестра я бы рекомендовал следующие курсы:
1. "Машинное обучение" - чтобы укрепить твои знания в этой области и изучить более продвинутые техники.
2. "Обработка данных" - для улучшения навыков работы с данными, что является ключевым в разработке ИИ-моделей.
3. "Алгоритмы и структуры данных" - чтобы укрепить фундаментальные знания, которые пригодятся в разработке эффективных решений.

Эти курсы помогут тебе заложить прочную основу для дальнейшего изучения и работы в области искусственного интеллекта.


### Сохраним индекс

In [218]:
index.storage_context.persist(persist_dir="../storage")